In [1]:
import h5py
import tabulate
import contextily as ctx
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import rasterio
import random
import math
from datetime import datetime, timedelta
# from IPython.display import HTML, display
from os import path
from shapely.geometry import Point
from matplotlib_scalebar.scalebar import ScaleBar
from matplotlib.lines import Line2D
from matplotlib import cm
from matplotlib.colors import ListedColormap
from scipy import stats
import time
import multiprocessing
from multiprocessing import Pool
from rasterio.plot import show
import functools
from osgeo import ogr
from osgeo import gdal
import os
cwd = os.getcwd()
print(cwd)
from glob import glob
from os import path

/home/liangm/.conda/envs/ea_work/lib/python3.7/site-packages/geopandas/_compat.py:53: UserWarning: The installed version of PyGEOS is too old (0.6 installed, 0.8 required), and thus GeoPandas will not use PyGEOS.
  UserWarning,


/gpfs/data1/duncansongp/amberliang/EA_data


## 1. read in samples, filter and separte by year

In [3]:
tileSamps=pd.read_csv('samples/all_gedi_tile173060plus3.csv')
print(tileSamps.shape)
tt= tileSamps[~tileSamps['agbd'].isna()]  #822686
tt2= tileSamps.loc[(tileSamps['algorithm_run_flag']==1) & (tileSamps['l2_quality_flag']==1) &
#                           (alldf['degrade_flag']==0) &
#                             (tileSamps['biome']==1)&
#                            (df['biome']!=13) & (df['biome']!=14) & (df['biome']!=0) &
                          (tileSamps['l4_quality_flag']==1) & tileSamps['agbd'].notnull()]
print(len(tt), len(tt2))

6511061 2637672


In [4]:
tt2.columns.tolist
tt2.delta_time

# datetime since
dt1 = datetime(2018, 1, 1, 0, 0, 0)
tt2['delta_time']=dt1+pd.to_timedelta(tt2['delta_time'],unit='s')
# type(sample['delta_time'][1])
# sample['delta_time'][1].date().year
tt2['year']=tt2['delta_time'].dt.year
tt2.year.value_counts()

/home/liangm/.conda/envs/ea_work/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/home/liangm/.conda/envs/ea_work/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


2020    993337
2019    830387
2021    813948
Name: year, dtype: int64

In [5]:
sample2019=tt2[tt2["year"]==2019]
sample2020=tt2[tt2["year"]==2020]
sample2021=tt2[tt2["year"]==2021]
# sample2020

sample2019.to_csv(cwd + '/' + "samples/all_gedi_tile173060plus3filtered_2019.csv", encoding='utf-8', index=False) 
sample2020.to_csv(cwd + '/' + "samples/all_gedi_tile173060plus3filtered_2020.csv", encoding='utf-8', index=False) 
sample2021.to_csv(cwd + '/' + "samples/all_gedi_tile173060plus3filtered_2021.csv", encoding='utf-8', index=False) 


## 2. extract landsat metrics by year 

In [7]:
topo = gdal.Open('/gpfs/data1/duncansongp/amberliang/EA_data/ancillary_vars/topo_stack.tif')
print(topo.GetRasterBand(3).GetDescription())

aspect


In [9]:
def get_raster_value(geo_x, geo_y, ds, band_index):
#     """Return raster value that corresponds to given coordinates."""
    forward_transform = ds.GetGeoTransform()
    pixel_width = forward_transform[1]
    pixel_height = -forward_transform[5]    
    reverse_transform = gdal.InvGeoTransform(forward_transform)
    pixel_coord = gdal.ApplyGeoTransform(reverse_transform, geo_x, geo_y)
    pixel_x = math.floor(pixel_coord[0])
    pixel_y = math.floor(pixel_coord[1])
    band = ds.GetRasterBand(band_index)
    val_arr = band.ReadAsArray(pixel_x, pixel_y, 1, 1) # Avoid reading the whole raster into memory - read 1x1 array
    if val_arr is None:
#         pass
#         print(out)
        return np.nan
    else:
        out=val_arr[0][0]
#         print(out)
        return out
def bname(ds, band_index):
    return  ds.GetRasterBand(band_index).GetDescription()

def getEx(pt, ds):
    from shapely.geometry import Polygon
    """ Return list of corner coordinates from a gdal Dataset """
    xmin, xpixel, _, ymax, _, ypixel = ds.GetGeoTransform()
    width, height = ds.RasterXSize, ds.RasterYSize
    xmax = xmin + width * xpixel
    ymin = ymax + height * ypixel
    
    poly=Polygon(((xmin, ymax), (xmax, ymax), (xmax, ymin), (xmin, ymin)))
    return poly.contains(pt)  #return true or false 

def ls_Extract(tileX, yr, biome, cornerInd):
    #tileX=tileX[0]
    print('tile', tileX)
    print('yr', yr)
    print('biome', biome)
    yr=str(yr)
    biome=str(biome)

    imageStack = "ls_lhs_medoid_"+tileX+"_"+yr+"_NBR_3by3_dur8_biome" + biome+ "-"+'*.tif'  ####!!![need to modify]!!!###
    out="/tilelevel_tile_"+tileX+"plus3_"+str(cornerInd)+ '_'+biome+'_'+yr+'_topo_lhs_nbr_33_dur8_nol2.csv'   ####!!![need to modify]!!!###

#     yr = yr[0]
#     tileX=tileX[0]
#     biome= biome[0]
    #preping the sample dataframe
    samples= pd.read_csv('samples/all_gedi_tile173060plus3filtered_'+yr+'.csv')   
#     samples =pd.read_csv("samples/"+"biome"+str(wwfbiome)+"_topo_training_l4a_l2b_lhs_nbr_33_dur8_empty_ndvi_"+yr+".csv")
    print('total samples', samples.shape)
    sample2020_min_1 = samples#.iloc[0:100]# [(samples['biome']==int(biome))]#cwd + '/' + "kibale_lite_for_gee_nicfi_"+yr+".csv") 
    print('sub samples', sample2020_min_1.shape)
    #     sample2020_min_1['shot_number']= sample2020_min_1['shot_number'].astype(str)
#     print(sample2020_min_1.dtypes)
    sample2020_min_1_gdf= gpd.GeoDataFrame(sample2020_min_1, 
                                           geometry=gpd.points_from_xy(sample2020_min_1.lon_lowestmode, sample2020_min_1.lat_lowestmode), 
                                           crs="EPSG:4326")
    #add in tile information to the sample gdf for easy filtering in later step
#     join = gpd.sjoin(sample2020_min_1_gdf, polys, how='left',op="within")
#     sample2020_min_1_gdf = join[join['Name'].notnull()]
#     sample2020_min_1_gdf=sample2020_min_1_gdf.drop(['descriptio', 'timestamp', 'begin', 'end', 'altitudeMo',
#            'tessellate', 'extrude', 'visibility', 'drawOrder', 'icon', 'geometry','index_right'], axis=1)
#     sample2020_min_1_gdf['Name']=sample2020_min_1_gdf['Name'].str.replace('_', '0')
#     biomeYrDF= gpd.GeoDataFrame(sample2020_min_1_gdf, geometry=gpd.points_from_xy(sample2020_min_1_gdf.lon_lowestmode, sample2020_min_1_gdf.lat_lowestmode), crs="EPSG:4326")

    sample2020_min_1_gdf_tile=sample2020_min_1_gdf
    print(sample2020_min_1_gdf_tile.shape)
#     sample2020_min_1_gdf_tile = biomeYrDF[biomeYrDF['Name'] == tileX]
#     year ="allVI_NBR_fit_"+yr+"_dur_10_PW_"+tileX+"_ALL"+'*.tif'
    pdir= "/gpfs/data1/duncansongp/amberliang/EA_data/landsat_comp/"+yr   #EA_data/gd_data/
    # pdir= "/gpfs/data1/duncansongp/amberliang/gd2/all_metrics_NBR_fitted"
    outdir = '/gpfs/data1/duncansongp/amberliang/EA_data/ls_ltr_training'#biome'+str(biome)+'_'+yr
    annual_pl= glob.glob(path.join(pdir, imageStack))[cornerInd]  
#     print("Tif files for each tile of each year", list(annual_pl))
    print('number of subset in tile: '+str(cornerInd))
    
    df = pd.DataFrame()
    for rcount in range(0,1):#range(0,len(annual_pl)):
        print('raster', rcount)
        src=gdal.Open(annual_pl)#gdal.Open(annual_pl[rcount])
        topo = gdal.Open('/gpfs/data1/duncansongp/amberliang/EA_data/ancillary_vars/topo_stack.tif')
#         i = 0
        while i < len(sample2020_min_1_gdf_tile):
            
            geo_x, geo_y = sample2020_min_1_gdf_tile.iloc[i]['lon_lowestmode'],sample2020_min_1_gdf_tile.iloc[i]['lat_lowestmode'] #coord_list[i][0], coord_list[i][1]
            geom =  sample2020_min_1_gdf_tile.iloc[i].geometry  #convert to point with geom for checking if its in the ras ex
            outlist = {}
            outlist['shot_number']=  sample2020_min_1_gdf_tile.iloc[i]['shot_number'].astype(str)
            outlist['agbd']=  sample2020_min_1_gdf_tile.iloc[i]['agbd']
            outlist['lon_lowestmode']=  sample2020_min_1_gdf_tile.iloc[i]['lon_lowestmode']
            outlist['lat_lowestmode']=  sample2020_min_1_gdf_tile.iloc[i]['lat_lowestmode']
            if getEx(geom, src):
#                 print('point in this subset')
                print(i)
                for b in range(0, src.RasterCount):
                    bind=b+1
                    out0=get_raster_value(geo_x, geo_y, ds=src, band_index=bind)
#                     out00= get_raster_value(geo_x, geo_y, ds=topo, band_index=bind)
#                     print("extracted",out0)
    #                 print(bname(src, bind))
                    if out0 is None:
                        out0=np.nan
    #                 print(out0)

                    outlist[bname(src, bind)]=out0
                for t in range(0, topo.RasterCount):
                    tind=t+1
                    
                    out00= get_raster_value(geo_x, geo_y, ds=topo, band_index=tind)
#                     print("extracted",out0)
    #                 print(bname(src, bind))
                    if out00 is None:
                        out0=np.nan
    #                 print(out0)

                    outlist[bname(topo, tind)]=out00
#                 print(outlist)
                if not (all(np.isnan(value) for value in list(outlist.values())[4:])):
#                     print(all(np.isnan(value) for value in list(outlist.values())[1:]))
                    df = df.append(outlist, ignore_index=True)
                i+=1
            else: 
                i+=1
    print(df.shape)
    print(df)
    
    if len(df)>0:
#         sample2020_min_1_gdf_tile_out= sample2020_min_1_gdf_tile.join(df, on="shot_number",how='left')
#         print(sample2020_min_1_gdf_tile_out.shape)
        df = df[(df['ftv_ndvi_fit']!=0) & (df['GDMAG']!=0)]
        print(str(len(df))+" out of total "+ str(len(sample2020_min_1_gdf_tile))+" were extracted")
        df.to_csv(outdir+out, index=False)
        return(df)
#         return df
    else:
        return df


In [ ]:
args = [(biomeTiles, yr, biome, cornerInd) for biomeTiles in biomeTiles for yr in yrs for cornerInd in corners]
print(len(args))
print(args)

In [6]:
# shp="AOI/EA_wgs_tiles.shp"
# polys = gpd.read_file(shp)
import glob
wwfbiome=1
biomeTiles= ['173061','173060', '173059', '172061','172060', '172059']#list(pd.read_csv("TSMBF_WGS2_tiles.csv")['Name'].str.replace('_', '0'))# ['169055']
yrs= ['2021', '2020','2019']
corners=range(0,12)
biome=str(wwfbiome)
# args = [(biomeTiles, yr, biome) for biomeTiles in biomeTiles]
args = [(biomeTiles, yr, biome, cornerInd) for biomeTiles in biomeTiles for yr in yrs for cornerInd in corners]
print(len(args))
print(args)
# parallel processing 
def main():
    tic = time.time()
    pool = Pool(processes=12)  # set the processes max number 3
    result = pool.starmap(ls_Extract, args)
    pool.terminate()
    pool.join()
    print(result)
    print('end')
    toc = time.time()
    print('Done in {:.4f} seconds'.format(toc-tic))

    
if __name__ == "__main__":
    main()

216
[('173061', '2021', '1', 0), ('173061', '2021', '1', 1), ('173061', '2021', '1', 2), ('173061', '2021', '1', 3), ('173061', '2021', '1', 4), ('173061', '2021', '1', 5), ('173061', '2021', '1', 6), ('173061', '2021', '1', 7), ('173061', '2021', '1', 8), ('173061', '2021', '1', 9), ('173061', '2021', '1', 10), ('173061', '2021', '1', 11), ('173061', '2020', '1', 0), ('173061', '2020', '1', 1), ('173061', '2020', '1', 2), ('173061', '2020', '1', 3), ('173061', '2020', '1', 4), ('173061', '2020', '1', 5), ('173061', '2020', '1', 6), ('173061', '2020', '1', 7), ('173061', '2020', '1', 8), ('173061', '2020', '1', 9), ('173061', '2020', '1', 10), ('173061', '2020', '1', 11), ('173061', '2019', '1', 0), ('173061', '2019', '1', 1), ('173061', '2019', '1', 2), ('173061', '2019', '1', 3), ('173061', '2019', '1', 4), ('173061', '2019', '1', 5), ('173061', '2019', '1', 6), ('173061', '2019', '1', 7), ('173061', '2019', '1', 8), ('173061', '2019', '1', 9), ('173061', '2019', '1', 10), ('173061',

In [10]:
# import glob
# wwfbiome=1
# biomeTiles= ['173060']#list(pd.read_csv("TSMBF_WGS2_tiles.csv")['Name'].str.replace('_', '0'))# ['169055']
# ## same extraction as below but using a looping method, good for testing 
# tic = time.time()
# for ind in range(0,12):
#     for yy in [2021]:
#         print(yy)
#         for tile in biomeTiles:
#             print(tile)
#     #         imageStack ="allVI_NBR_fit_"+yr+"_dur_10_PW_"+tileX+"_ALL"+'*.tif'
#     #         imageStack = "ls_lhs_medoid_"+tile+"_"+yr+"_NBR_3by3_dur8_biome" + str(wwfbiome)+ "-"+'*.tif'  ####!!![need to modify]!!!###
#     #         outName="/tile_"+tile+ '_'+biome+'_'+yr+'_topo_lhs_nbr_33_dur8.csv'   ####!!![need to modify]!!!###
#             annual_tile_samp= ls_Extract(tileX=tile, yr=yy, biome = wwfbiome, cornerInd= ind)
# toc = time.time()
# print('Done in {:.4f} seconds'.format(toc-tic))

2021
173060
tile 173060
yr 2021
biome 1
total samples (113265, 53)
sub samples (100, 53)
(100, 53)
number of subset in tile: 0
raster 0
7
12
13
14
15
16
17
18
19
20
21
22
(12, 104)
    ADDUR  ADMAG  ADMG5  ADRE  ADRE5  ADROC  ADVA5  ADVAL     BDDUR     BDMAG  \
0     0.0    0.0    0.0   0.0    0.0    0.0    0.0    0.0  0.000000  0.000000   
1     0.0    0.0    0.0   0.0    0.0    0.0    0.0    0.0  0.000000  0.000000   
2     0.0    0.0    0.0   0.0    0.0    0.0    0.0    0.0  0.000000  0.000000   
3     0.0    0.0    0.0   0.0    0.0    0.0    0.0    0.0  0.061224 -0.063918   
4     0.0    0.0    0.0   0.0    0.0    0.0    0.0    0.0  0.000000  0.000000   
5     0.0    0.0    0.0   0.0    0.0    0.0    0.0    0.0  0.000000  0.000000   
6     0.0    0.0    0.0   0.0    0.0    0.0    0.0    0.0  0.000000  0.000000   
7     0.0    0.0    0.0   0.0    0.0    0.0    0.0    0.0  0.000000  0.000000   
8     0.0    0.0    0.0   0.0    0.0    0.0    0.0    0.0  0.000000  0.000000   
9     0.0

total samples (113265, 53)
sub samples (100, 53)
(100, 53)
number of subset in tile: 4
raster 0
(0, 0)
Empty DataFrame
Columns: []
Index: []
2021
173060
tile 173060
yr 2021
biome 1
total samples (113265, 53)
sub samples (100, 53)
(100, 53)
number of subset in tile: 5
raster 0
(0, 0)
Empty DataFrame
Columns: []
Index: []
2021
173060
tile 173060
yr 2021
biome 1
total samples (113265, 53)
sub samples (100, 53)
(100, 53)
number of subset in tile: 6
raster 0
(0, 0)
Empty DataFrame
Columns: []
Index: []
2021
173060
tile 173060
yr 2021
biome 1
total samples (113265, 53)
sub samples (100, 53)
(100, 53)
number of subset in tile: 7
raster 0
(0, 0)
Empty DataFrame
Columns: []
Index: []
2021
173060
tile 173060
yr 2021
biome 1
total samples (113265, 53)
sub samples (100, 53)
(100, 53)
number of subset in tile: 8
raster 0
(0, 0)
Empty DataFrame
Columns: []
Index: []
2021
173060
tile 173060
yr 2021
biome 1
total samples (113265, 53)
sub samples (100, 53)
(100, 53)
number of subset in tile: 9
raster 0

## 3. combine extracted files for tile 173060

In [8]:
extracted=  glob('ls_ltr_training/tile168064_filtered_l4a_ls_output_corner/*.csv')#*plus3_*_nol2.csv')  
print(len(extracted))
tileEx= pd.DataFrame([])
for ef in extracted:
    print(ef)
    ex= pd.read_csv(ef)
    tileEx = tileEx.append(ex, ignore_index=True)
    print(tileEx.shape)
##export the ls_l4a to a csv file 
# tileEx.to_csv('ls_ltr_training/tile169055_l4a_topo_ls_lhs_nbr_33_dur8_highval.csv')

27
ls_ltr_training/tile168064_filtered_l4a_ls_output_corner/tilelevel_tile_168064_cor6_biom1_2019_topo_lhs_nbr_33_dur8.csv
(14072, 104)
ls_ltr_training/tile168064_filtered_l4a_ls_output_corner/tilelevel_tile_168064_cor1_biom1_2021_topo_lhs_nbr_33_dur8.csv
(64665, 104)
ls_ltr_training/tile168064_filtered_l4a_ls_output_corner/tilelevel_tile_168064_cor7_biom1_2020_topo_lhs_nbr_33_dur8.csv
(84641, 104)
ls_ltr_training/tile168064_filtered_l4a_ls_output_corner/tilelevel_tile_168064_cor10_biom1_2020_topo_lhs_nbr_33_dur8.csv
(93605, 104)
ls_ltr_training/tile168064_filtered_l4a_ls_output_corner/tilelevel_tile_168064_cor5_biom1_2021_topo_lhs_nbr_33_dur8.csv
(111318, 104)
ls_ltr_training/tile168064_filtered_l4a_ls_output_corner/tilelevel_tile_168064_cor13_biom1_2021_topo_lhs_nbr_33_dur8.csv
(145991, 104)
ls_ltr_training/tile168064_filtered_l4a_ls_output_corner/tilelevel_tile_168064_cor5_biom1_2019_topo_lhs_nbr_33_dur8.csv
(172248, 104)
ls_ltr_training/tile168064_filtered_l4a_ls_output_corner/tile

In [9]:
#for desa combine tile 168051 + 168050 
t1 = tileEx
# t1= pd.read_csv('ls_ltr_training/tile169055_l4a_topo_ls_lhs_nbr_33_dur8_highval.csv')
# t2= pd.read_csv('ls_ltr_training/tile168051_l4a_topo_ls_lhs_nbr_33_dur8_highval.csv')
# t12= t1.append(t2, ignore_index=True)

In [10]:
l4a_orig=  glob('l4a_csv/dodoma1/*.csv')#*plus3_*_nol2.csv')  
print(len(l4a_orig))
l4a_orig_all= pd.DataFrame([])
for ef in l4a_orig:
    print(ef)
    ex= pd.read_csv(ef)
    l4a_orig_all = l4a_orig_all.append(ex, ignore_index=True)
    print(l4a_orig_all.shape)

254
l4a_csv/dodoma1/GEDI04_A_2020331101959_O11085_01_T08287_02_002_02_V002_sub.h5.csv
(2457, 51)
l4a_csv/dodoma1/GEDI04_A_2022223035610_O20739_01_T00208_02_003_01_V002_sub.h5.csv
(31000, 51)
l4a_csv/dodoma1/GEDI04_A_2021134031240_O13700_04_T09748_02_002_02_V002_sub.h5.csv
(37467, 51)
l4a_csv/dodoma1/GEDI04_A_2022200125213_O20388_01_T10475_02_003_02_V002_sub.h5.csv
(66006, 51)
l4a_csv/dodoma1/GEDI04_A_2021332204942_O16780_04_T08218_02_002_02_V002_sub.h5.csv
(69476, 51)
l4a_csv/dodoma1/GEDI04_A_2021016140121_O11878_01_T10475_02_002_02_V002_sub.h5.csv
(98074, 51)
l4a_csv/dodoma1/GEDI04_A_2021314160444_O16498_01_T06818_02_002_02_V002_sub.h5.csv
(108044, 51)
l4a_csv/dodoma1/GEDI04_A_2021322130148_O16620_01_T07782_02_002_02_V002_sub.h5.csv
(136637, 51)
l4a_csv/dodoma1/GEDI04_A_2021157175543_O14066_04_T06443_02_002_02_V002_sub.h5.csv
(162465, 51)
l4a_csv/dodoma1/GEDI04_A_2020092085413_O07379_01_T01172_02_002_02_V002_sub.h5.csv
(165782, 51)
l4a_csv/dodoma1/GEDI04_A_2020169140154_O08576_04_T004

(1447343, 51)
l4a_csv/dodoma1/GEDI04_A_2022127174153_O19259_01_T00361_02_003_02_V002_sub.h5.csv
(1475910, 51)
l4a_csv/dodoma1/GEDI04_A_2022233112857_O20899_04_T09901_02_003_01_V002_sub.h5.csv
(1476429, 51)
l4a_csv/dodoma1/GEDI04_A_2022097054625_O18786_01_T06053_02_003_01_V002_sub.h5.csv
(1504956, 51)
l4a_csv/dodoma1/GEDI04_A_2022309054123_O22074_04_T03949_02_003_01_V002_sub.h5.csv
(1509509, 51)
l4a_csv/dodoma1/GEDI04_A_2022321005220_O22257_04_T00904_02_003_01_V002_sub.h5.csv
(1530081, 51)
l4a_csv/dodoma1/GEDI04_A_2019274205457_O04549_04_T02526_02_002_02_V002_sub.h5.csv
(1530276, 51)
l4a_csv/dodoma1/GEDI04_A_2019260023538_O04320_04_T05479_02_002_02_V002_sub.h5.csv
(1540467, 51)
l4a_csv/dodoma1/GEDI04_A_2021145223400_O13883_04_T08172_02_002_02_V002_sub.h5.csv
(1555869, 51)
l4a_csv/dodoma1/GEDI04_A_2021326113016_O16681_01_T04630_02_002_02_V002_sub.h5.csv
(1584435, 51)
l4a_csv/dodoma1/GEDI04_A_2021052000454_O12427_01_T11286_02_002_02_V002_sub.h5.csv
(1593585, 51)
l4a_csv/dodoma1/GEDI04_A_2

(2993776, 51)
l4a_csv/dodoma1/GEDI04_A_2020163044714_O08477_01_T00514_02_002_02_V002_sub.h5.csv
(3022358, 51)
l4a_csv/dodoma1/GEDI04_A_2020167031239_O08538_01_T00820_02_002_02_V002_sub.h5.csv
(3050797, 51)
l4a_csv/dodoma1/GEDI04_A_2020157184539_O08393_04_T01562_02_002_02_V002_sub.h5.csv
(3068643, 51)
l4a_csv/dodoma1/GEDI04_A_2022051000232_O18069_01_T06359_02_003_02_V002_sub.h5.csv
(3097242, 51)
l4a_csv/dodoma1/GEDI04_A_2019335203545_O05495_04_T02633_02_002_02_V002_sub.h5.csv
(3108949, 51)
l4a_csv/dodoma1/GEDI04_A_2019216081823_O03641_01_T04171_02_002_02_V002_sub.h5.csv
(3123774, 51)
l4a_csv/dodoma1/GEDI04_A_2021253041510_O15545_04_T07407_02_002_02_V002_sub.h5.csv
(3149791, 51)
l4a_csv/dodoma1/GEDI04_A_2022045140256_O17985_04_T02679_02_003_01_V002_sub.h5.csv
(3155773, 51)
l4a_csv/dodoma1/GEDI04_A_2020349145551_O11367_04_T08218_02_002_02_V002_sub.h5.csv
(3158013, 51)
l4a_csv/dodoma1/GEDI04_A_2022249172420_O21151_01_T10322_02_003_01_V002_sub.h5.csv
(3186575, 51)
l4a_csv/dodoma1/GEDI04_A_2

In [11]:
dt1 = datetime(2018, 1, 1, 0, 0, 0)
l4a_orig_all['delta_time']=dt1+pd.to_timedelta(l4a_orig_all['delta_time'],unit='s')
# type(sample['delta_time'][1])
# sample['delta_time'][1].date().year
l4a_orig_all['year']=l4a_orig_all['delta_time'].dt.year

In [12]:
tile_samps_l4a = t1.merge(l4a_orig_all,on='shot_number',how='left')
print(tile_samps_l4a.shape, t1.shape)
tile_samps_l4a.to_csv('ls_ltr_training/tile168064_l4aAll_topo_ls_lhs_nbr_33_dur8_highval.csv')

(774802, 155) (774802, 104)


In [13]:
## skip the steps below and left join with l2b products 
import json
from pandas import json_normalize

# tt2 = json.loads('l2b_subset/desa_t2/GEDI02_B_2019131042706_O02319_03_T02724_02_003_01_V002.json')

path=r'dodoma1_l2b_042019_to_122022.json'

with open(path) as data_file:
    data = json.load(data_file)
    df =json_normalize([i['properties'] for i in data['features']])

print(df)

tile_samps_l2bl4a = tile_samps_l4a.merge(df,on='shot_number',how='left')
print(tile_samps_l2bl4a.shape, df.shape)

tile_samps_l2bl4a.to_csv('ls_ltr_training/tile168064_l24a_all_topo_ls_lhs_nbr_33_dur8_highval.csv')

             BEAM         shot_number  Latitude  Longitude  index  \
0        BEAM0000  120380000400455234 -5.661861  35.291067  14128   
1        BEAM0000  120380000400455235 -5.662282  35.291366  14129   
2        BEAM0000  120380000400455236 -5.662703  35.291665  14130   
3        BEAM0000  120380000400455237 -5.663124  35.291964  14131   
4        BEAM0000  120380000400455238 -5.663545  35.292262  14132   
...           ...                 ...       ...        ...    ...   
4437360  BEAM1011   84161100100094881 -5.030345  36.663791  94880   
4437361  BEAM1011   84161100100094882 -5.029929  36.664086  94881   
4437362  BEAM1011   84161100100094883 -5.029505  36.664388  94882   
4437363  BEAM1011   84161100100094884 -5.028583  36.665120  94883   
4437364  BEAM1011   84161100100094885 -5.028163  36.665419  94884   

         algorithmrun_flag        cover    cover_z_0    cover_z_1  \
0                        0 -9999.000000 -9999.000000 -9999.000000   
1                        0 -9999.

## below is for if there are multiple tiles

In [37]:
# for the 4_tile compilation

l4a_orig=  glob('l4a_csv/desa_t4/*.csv')#*plus3_*_nol2.csv')  
print(len(l4a_orig))
l4a_orig_all= pd.DataFrame([])
for ef in l4a_orig:
    print(ef)
    ex= pd.read_csv(ef)
    l4a_orig_all = l4a_orig_all.append(ex, ignore_index=True)
    print(l4a_orig_all.shape)

520
l4a_csv/desa_t4/GEDI04_A_2020118225212_O07791_02_T05426_02_002_02_V002_sub.h5.csv
(10937, 51)
l4a_csv/desa_t4/GEDI04_A_2020180222749_O08752_02_T01356_02_002_02_V002_sub.h5.csv
(69438, 51)
l4a_csv/desa_t4/GEDI04_A_2020345053917_O11299_02_T08517_02_002_02_V002_sub.h5.csv
(87335, 51)
l4a_csv/desa_t4/GEDI04_A_2020232020412_O09545_02_T00040_02_002_02_V002_sub.h5.csv
(87335, 51)
l4a_csv/desa_t4/GEDI04_A_2021141131621_O13815_02_T07201_02_002_02_V002_sub.h5.csv
(145846, 51)
l4a_csv/desa_t4/GEDI04_A_2020141001722_O08133_03_T00123_02_002_02_V002_sub.h5.csv
(177972, 51)
l4a_csv/desa_t4/GEDI04_A_2020130180929_O07974_02_T00040_02_002_02_V002_sub.h5.csv
(177972, 51)
l4a_csv/desa_t4/GEDI04_A_2019286051318_O04725_02_T04508_02_002_02_V002_sub.h5.csv
(236504, 51)
l4a_csv/desa_t4/GEDI04_A_2021069035627_O12693_03_T09931_02_002_02_V002_sub.h5.csv
(264996, 51)
l4a_csv/desa_t4/GEDI04_A_2019255032806_O04243_03_T02617_02_002_02_V002_sub.h5.csv
(324886, 51)
l4a_csv/desa_t4/GEDI04_A_2020313180654_O10811_02_T

(1975104, 51)
l4a_csv/desa_t4/GEDI04_A_2020019003541_O06242_03_T03780_02_002_02_V002_sub.h5.csv
(1985623, 51)
l4a_csv/desa_t4/GEDI04_A_2020285151034_O10375_03_T06320_02_002_02_V002_sub.h5.csv
(1985623, 51)
l4a_csv/desa_t4/GEDI04_A_2019349140945_O05708_03_T02969_02_002_02_V002_sub.h5.csv
(2019232, 51)
l4a_csv/desa_t4/GEDI04_A_2021335183059_O16825_03_T05815_02_002_02_V002_sub.h5.csv
(2052011, 51)
l4a_csv/desa_t4/GEDI04_A_2020341071252_O11238_02_T11363_02_002_02_V002_sub.h5.csv
(2067855, 51)
l4a_csv/desa_t4/GEDI04_A_2021256155333_O15599_02_T05977_02_002_02_V002_sub.h5.csv
(2104356, 51)
l4a_csv/desa_t4/GEDI04_A_2019294015833_O04847_02_T01463_02_002_02_V002_sub.h5.csv
(2104356, 51)
l4a_csv/desa_t4/GEDI04_A_2020141141317_O08142_02_T03942_02_002_02_V002_sub.h5.csv
(2109097, 51)
l4a_csv/desa_t4/GEDI04_A_2020137154738_O08081_02_T02519_02_002_02_V002_sub.h5.csv
(2109097, 51)
l4a_csv/desa_t4/GEDI04_A_2020309054524_O10741_03_T09931_02_002_02_V002_sub.h5.csv
(2137203, 51)
l4a_csv/desa_t4/GEDI04_A_2

(3854192, 51)
l4a_csv/desa_t4/GEDI04_A_2019222063408_O03733_02_T04355_02_002_02_V002_sub.h5.csv
(3912714, 51)
l4a_csv/desa_t4/GEDI04_A_2020325132610_O10994_02_T08364_02_002_02_V002_sub.h5.csv
(3922523, 51)
l4a_csv/desa_t4/GEDI04_A_2020206221730_O09155_03_T00781_02_002_02_V002_sub.h5.csv
(3925772, 51)
l4a_csv/desa_t4/GEDI04_A_2019263001316_O04365_03_T03688_02_002_02_V002_sub.h5.csv
(3950589, 51)
l4a_csv/desa_t4/GEDI04_A_2021263225527_O15712_03_T08508_02_002_02_V002_sub.h5.csv
(3976223, 51)
l4a_csv/desa_t4/GEDI04_A_2020161062058_O08447_02_T04860_02_002_02_V002_sub.h5.csv
(4013541, 51)
l4a_csv/desa_t4/GEDI04_A_2021351122005_O17069_03_T11048_02_002_02_V002_sub.h5.csv
(4026250, 51)
l4a_csv/desa_t4/GEDI04_A_2019251190142_O04191_02_T01555_02_002_02_V002_sub.h5.csv
(4049306, 51)
l4a_csv/desa_t4/GEDI04_A_2021172105734_O14294_03_T09732_02_002_02_V002_sub.h5.csv
(4107759, 51)
l4a_csv/desa_t4/GEDI04_A_2019167042859_O02878_02_T05166_02_002_02_V002_sub.h5.csv
(4166205, 51)
l4a_csv/desa_t4/GEDI04_A_2

(5876837, 51)
l4a_csv/desa_t4/GEDI04_A_2020247194928_O09789_02_T05885_02_002_02_V002_sub.h5.csv
(5876837, 51)
l4a_csv/desa_t4/GEDI04_A_2021197145410_O14684_02_T10307_02_002_02_V002_sub.h5.csv
(5876837, 51)
l4a_csv/desa_t4/GEDI04_A_2021196014415_O14660_03_T07391_02_002_02_V002_sub.h5.csv
(5912095, 51)
l4a_csv/desa_t4/GEDI04_A_2021332095915_O16773_02_T06941_02_002_02_V002_sub.h5.csv
(5920619, 51)
l4a_csv/desa_t4/GEDI04_A_2019131182115_O02328_02_T02580_02_002_02_V002_sub.h5.csv
(5933639, 51)
l4a_csv/desa_t4/GEDI04_A_2019120225018_O02160_02_T02320_02_002_02_V002_sub.h5.csv
(5992094, 51)
l4a_csv/desa_t4/GEDI04_A_2021275181931_O15895_03_T09319_02_002_02_V002_sub.h5.csv
(5997505, 51)
l4a_csv/desa_t4/GEDI04_A_2020023125623_O06312_02_T00943_02_002_02_V002_sub.h5.csv
(5997505, 51)
l4a_csv/desa_t4/GEDI04_A_2019203140742_O03443_02_T04401_02_002_02_V002_sub.h5.csv
(6022674, 51)
l4a_csv/desa_t4/GEDI04_A_2021092183943_O13059_03_T06473_02_002_02_V002_sub.h5.csv
(6026681, 51)
l4a_csv/desa_t4/GEDI04_A_2

(7952146, 51)
l4a_csv/desa_t4/GEDI04_A_2021149101056_O13937_02_T01203_02_002_02_V002_sub.h5.csv
(8010579, 51)
l4a_csv/desa_t4/GEDI04_A_2020015021045_O06181_03_T03627_02_002_02_V002_sub.h5.csv
(8015274, 51)
l4a_csv/desa_t4/GEDI04_A_2019138014405_O02426_03_T03841_02_002_02_V002_sub.h5.csv
(8034415, 51)
l4a_csv/desa_t4/GEDI04_A_2020160162505_O08438_03_T03734_02_002_02_V002_sub.h5.csv
(8091606, 51)
l4a_csv/desa_t4/GEDI04_A_2021097162054_O13135_03_T05723_02_002_02_V002_sub.h5.csv
(8094379, 51)
l4a_csv/desa_t4/GEDI04_A_2019196024454_O03327_03_T03535_02_002_02_V002_sub.h5.csv
(8124970, 51)
l4a_csv/desa_t4/GEDI04_A_2021125053205_O13562_03_T09227_02_002_02_V002_sub.h5.csv
(8154745, 51)
l4a_csv/desa_t4/GEDI04_A_2021229023416_O15172_02_T09236_02_002_02_V002_sub.h5.csv
(8185694, 51)
l4a_csv/desa_t4/GEDI04_A_2020164145030_O08499_03_T01041_02_002_02_V002_sub.h5.csv
(8247236, 51)
l4a_csv/desa_t4/GEDI04_A_2019117095234_O02105_03_T01760_02_002_02_V002_sub.h5.csv
(8247236, 51)
l4a_csv/desa_t4/GEDI04_A_2

(9891756, 51)
l4a_csv/desa_t4/GEDI04_A_2019334195023_O05479_03_T01806_02_002_02_V002_sub.h5.csv
(9933114, 51)
l4a_csv/desa_t4/GEDI04_A_2021320143358_O16590_02_T07859_02_002_02_V002_sub.h5.csv
(9991762, 51)
l4a_csv/desa_t4/GEDI04_A_2020262142259_O10018_02_T06482_02_002_02_V002_sub.h5.csv
(9991762, 51)
l4a_csv/desa_t4/GEDI04_A_2019323143617_O05305_02_T05671_02_002_02_V002_sub.h5.csv
(10008674, 51)
l4a_csv/desa_t4/GEDI04_A_2021308191029_O16407_02_T09741_02_002_02_V002_sub.h5.csv
(10067287, 51)
l4a_csv/desa_t4/GEDI04_A_2021125192813_O13571_02_T10353_02_002_02_V002_sub.h5.csv
(10123601, 51)
l4a_csv/desa_t4/GEDI04_A_2019188200020_O03214_02_T04095_02_002_02_V002_sub.h5.csv
(10136481, 51)
l4a_csv/desa_t4/GEDI04_A_2021084214522_O12937_03_T06626_02_002_02_V002_sub.h5.csv
(10146302, 51)
l4a_csv/desa_t4/GEDI04_A_2019345154650_O05647_03_T05356_02_002_02_V002_sub.h5.csv
(10159719, 51)
l4a_csv/desa_t4/GEDI04_A_2020039064236_O06556_02_T03789_02_002_02_V002_sub.h5.csv
(10164682, 51)
l4a_csv/desa_t4/GED

(11949905, 51)
l4a_csv/desa_t4/GEDI04_A_2019134172439_O02374_02_T04707_02_002_02_V002_sub.h5.csv
(11991452, 51)
l4a_csv/desa_t4/GEDI04_A_2019174015518_O02985_02_T03483_02_002_02_V002_sub.h5.csv
(11991452, 51)
l4a_csv/desa_t4/GEDI04_A_2020180083158_O08743_03_T04805_02_002_02_V002_sub.h5.csv
(12026677, 51)
l4a_csv/desa_t4/GEDI04_A_2021216171549_O14980_03_T05876_02_002_02_V002_sub.h5.csv
(12026677, 51)
l4a_csv/desa_t4/GEDI04_A_2020016151935_O06205_02_T05579_02_002_02_V002_sub.h5.csv
(12029091, 51)


In [36]:
dt1 = datetime(2018, 1, 1, 0, 0, 0)
l4a_orig_all['delta_time']=dt1+pd.to_timedelta(l4a_orig_all['delta_time'],unit='s')
# type(sample['delta_time'][1])
# sample['delta_time'][1].date().year
l4a_orig_all['year']=l4a_orig_all['delta_time'].dt.year

In [38]:
#run this if 
t3= pd.read_csv('ls_ltr_training/tile169050_l4a_topo_ls_lhs_nbr_33_dur8_highval.csv')
t4= pd.read_csv('ls_ltr_training/tile169051_l4a_topo_ls_lhs_nbr_33_dur8_highval.csv')
t14= t12.append(t3, ignore_index=True).append(t4, ignore_index=True)
print(t14.shape)

(1646782, 105)


In [39]:
tile_samps_l4a = t14.merge(l4a_orig_all,on='shot_number',how='left')
print(tile_samps_l4a.shape, df.shape)
tile_samps_l4a.to_csv('ls_ltr_training/tile168050+168051+169050+169051_l4aAll_topo_ls_lhs_nbr_33_dur8_highval.csv')

(1646782, 155) (17132061, 128)


In [40]:
path=r'desa_tile4_l2b_042019_to_122022.json'

with open(path) as data_file:
    data = json.load(data_file)
    df =json_normalize([i['properties'] for i in data['features']])

print(df)

tile_samps_l2bl4a = tile_samps_l4a.merge(df,on='shot_number',how='left')
print(tile_samps_l2bl4a.shape, df.shape)

tile_samps_l2bl4a.to_csv('ls_ltr_training/tile168050+168051+169050+169051_l24a_all_topo_ls_lhs_nbr_33_dur8_highval.csv')

              BEAM         shot_number   Latitude  Longitude   index  \
0         BEAM0000   28690000300438723  15.387998  38.242913  125358   
1         BEAM0000   28690000300438724  15.387586  38.243234  125359   
2         BEAM0000   28690000300438725  15.387174  38.243556  125360   
3         BEAM0000   28690000300438726  15.386762  38.243877  125361   
4         BEAM0000   28690000300438727  15.386350  38.244198  125362   
...            ...                 ...        ...        ...     ...   
17132056  BEAM1011  137451100300313084  13.461008  41.167731  135696   
17132057  BEAM1011  137451100300313085  13.460591  41.168046  135697   
17132058  BEAM1011  137451100300313086  13.460182  41.168361  135698   
17132059  BEAM1011  137451100300313087  13.459765  41.168677  135699   
17132060  BEAM1011  137451100300313088  13.459353  41.168992  135700   

          algorithmrun_flag        cover    cover_z_0  cover_z_1  cover_z_2  \
0                         1     0.022164     0.022164   

In [62]:
# ##compare with the original samps dimension: the diff can be from cloud or other masked pixels, or edge of scene
# tileSamps=pd.read_csv('samples/gedi_samples_biome1_2021_highval.csv')
# tileSamps2=pd.read_csv('samples/gedi_samples_biome1_2020_highval.csv')
# tileSamps3=pd.read_csv('samples/gedi_samples_biome1_2019_highval.csv')
# print(len(tiZ  leSamps)+len(tileSamps2)+len(tileSamps3))
# print(tileEx.shape)

2649875
(3229769, 104)


## 4. join the combined files with l2b 
### this uses the samples_l2b data which already got pre-matched with the l4a data in step6 (or step 9a)


In [66]:
l2bf= glob.glob('samples/sample_l2b/*_highval.csv')
dt1 = datetime(2018, 1, 1, 0, 0, 0)
l2b_samps2020= pd.DataFrame([])
l2b_samps2021= pd.DataFrame([])
l2b_samps2019= pd.DataFrame([])
for x in range(0,len(l2bf)):
    print(x, 'in', len(l2bf))
    l2b_samps=pd.read_csv(l2bf[x])
    l2b_samps['geolocation_delta_time']=dt1+pd.to_timedelta(l2b_samps['geolocation_delta_time'],unit='s')
    l2b_samps['year']=l2b_samps['geolocation_delta_time'].dt.year
    print(l2b_samps.year.value_counts())
    if l2b_samps.year.unique()[0]== 2021:
        l2b_samps2021 = l2b_samps2021.append(l2b_samps, ignore_index=True)
    elif l2b_samps.year.unique()[0]== 2020:
        l2b_samps2020 = l2b_samps2020.append(l2b_samps, ignore_index=True)
    elif l2b_samps.year.unique()[0]== 2019:
        l2b_samps2019 = l2b_samps2019.append(l2b_samps, ignore_index=True)
        
print('number of observtaions for 2019, 2020, 2021:', len(l2b_samps2019),len(l2b_samps2020),len(l2b_samps2021) )
#     # Appending this single-row-DataFrame to the `pre_dfs` list
#     l2b_samps = l2b_samps.append(l2b, ignore_index=True)
          

0 in 1823
2020    1777
Name: year, dtype: int64
1 in 1823
2020    470
Name: year, dtype: int64
2 in 1823
2020    4543
Name: year, dtype: int64
3 in 1823
2021    3114
Name: year, dtype: int64
4 in 1823
2020    1100
Name: year, dtype: int64
5 in 1823
2020    508
Name: year, dtype: int64
6 in 1823
2019    3861
Name: year, dtype: int64
7 in 1823
2019    1892
Name: year, dtype: int64
8 in 1823
2021    305
Name: year, dtype: int64
9 in 1823
2019    75
Name: year, dtype: int64
10 in 1823
2019    3699
Name: year, dtype: int64
11 in 1823
2020    1050
Name: year, dtype: int64
12 in 1823
2021    3716
Name: year, dtype: int64
13 in 1823
2020    663
Name: year, dtype: int64
14 in 1823
2019    1731
Name: year, dtype: int64
15 in 1823
2020    151
Name: year, dtype: int64
16 in 1823
2021    19
Name: year, dtype: int64
17 in 1823
2020    828
Name: year, dtype: int64
18 in 1823
2020    412
Name: year, dtype: int64
19 in 1823
2019    1394
Name: year, dtype: int64
20 in 1823
2020    770
Name: year, dtype:

170 in 1823
2020    3457
Name: year, dtype: int64
171 in 1823
2020    2269
Name: year, dtype: int64
172 in 1823
2021    970
Name: year, dtype: int64
173 in 1823
2019    1195
Name: year, dtype: int64
174 in 1823
2020    562
Name: year, dtype: int64
175 in 1823
2021    685
Name: year, dtype: int64
176 in 1823
2021    4214
Name: year, dtype: int64
177 in 1823
2019    1224
Name: year, dtype: int64
178 in 1823
2021    5466
Name: year, dtype: int64
179 in 1823
2021    4323
Name: year, dtype: int64
180 in 1823
2020    1606
Name: year, dtype: int64
181 in 1823
2020    559
Name: year, dtype: int64
182 in 1823
2020    1355
Name: year, dtype: int64
183 in 1823
2019    4886
Name: year, dtype: int64
184 in 1823
2019    3
Name: year, dtype: int64
185 in 1823
2020    627
Name: year, dtype: int64
186 in 1823
2019    465
Name: year, dtype: int64
187 in 1823
2021    2080
Name: year, dtype: int64
188 in 1823
2020    1472
Name: year, dtype: int64
189 in 1823
2019    5662
Name: year, dtype: int64
190 in 18

337 in 1823
2019    1455
Name: year, dtype: int64
338 in 1823
2021    2012
Name: year, dtype: int64
339 in 1823
2020    274
Name: year, dtype: int64
340 in 1823
2020    2
Name: year, dtype: int64
341 in 1823
2019    878
Name: year, dtype: int64
342 in 1823
2020    2873
Name: year, dtype: int64
343 in 1823
2020    2908
Name: year, dtype: int64
344 in 1823
2020    2234
Name: year, dtype: int64
345 in 1823
2021    840
Name: year, dtype: int64
346 in 1823
2021    2326
Name: year, dtype: int64
347 in 1823
2020    1024
Name: year, dtype: int64
348 in 1823
2020    83
Name: year, dtype: int64
349 in 1823
2021    1773
Name: year, dtype: int64
350 in 1823
2019    187
Name: year, dtype: int64
351 in 1823
2021    509
Name: year, dtype: int64
352 in 1823
2021    1574
Name: year, dtype: int64
353 in 1823
2020    473
Name: year, dtype: int64
354 in 1823
2021    3858
Name: year, dtype: int64
355 in 1823
2021    285
Name: year, dtype: int64
356 in 1823
2021    201
Name: year, dtype: int64
357 in 1823
2

504 in 1823
2020    342
Name: year, dtype: int64
505 in 1823
2021    19
Name: year, dtype: int64
506 in 1823
2021    45
Name: year, dtype: int64
507 in 1823
2021    1305
Name: year, dtype: int64
508 in 1823
2019    769
Name: year, dtype: int64
509 in 1823
2020    1046
Name: year, dtype: int64
510 in 1823
2019    756
Name: year, dtype: int64
511 in 1823
2020    4
Name: year, dtype: int64
512 in 1823
2021    5
Name: year, dtype: int64
513 in 1823
2020    2649
Name: year, dtype: int64
514 in 1823
2020    1070
Name: year, dtype: int64
515 in 1823
2021    2035
Name: year, dtype: int64
516 in 1823
2019    2002
Name: year, dtype: int64
517 in 1823
2020    69
Name: year, dtype: int64
518 in 1823
2021    2341
Name: year, dtype: int64
519 in 1823
2020    128
Name: year, dtype: int64
520 in 1823
2019    1423
Name: year, dtype: int64
521 in 1823
2020    451
Name: year, dtype: int64
522 in 1823
2021    215
Name: year, dtype: int64
523 in 1823
2020    76
Name: year, dtype: int64
524 in 1823
2019    

671 in 1823
2020    1946
Name: year, dtype: int64
672 in 1823
2021    5388
Name: year, dtype: int64
673 in 1823
2019    2766
Name: year, dtype: int64
674 in 1823
2020    70
Name: year, dtype: int64
675 in 1823
2021    5033
Name: year, dtype: int64
676 in 1823
2020    5817
Name: year, dtype: int64
677 in 1823
2020    1404
Name: year, dtype: int64
678 in 1823
2020    876
Name: year, dtype: int64
679 in 1823
2021    4665
Name: year, dtype: int64
680 in 1823
2021    2217
Name: year, dtype: int64
681 in 1823
2020    2958
Name: year, dtype: int64
682 in 1823
2021    2
Name: year, dtype: int64
683 in 1823
2020    1341
Name: year, dtype: int64
684 in 1823
2021    1136
Name: year, dtype: int64
685 in 1823
2020    378
Name: year, dtype: int64
686 in 1823
2021    1025
Name: year, dtype: int64
687 in 1823
2021    745
Name: year, dtype: int64
688 in 1823
2021    5874
Name: year, dtype: int64
689 in 1823
2020    1774
Name: year, dtype: int64
690 in 1823
2019    1252
Name: year, dtype: int64
691 in 1

...

1167 in 1823
2021    189
Name: year, dtype: int64
1168 in 1823
2019    295
Name: year, dtype: int64
1169 in 1823
2020    4591
Name: year, dtype: int64
1170 in 1823
2020    3791
Name: year, dtype: int64
1171 in 1823
2020    1878
Name: year, dtype: int64
1172 in 1823
2021    153
Name: year, dtype: int64
1173 in 1823
2019    1637
Name: year, dtype: int64
1174 in 1823
2020    4785
Name: year, dtype: int64
1175 in 1823
2021    898
Name: year, dtype: int64
1176 in 1823
2021    128
Name: year, dtype: int64
1177 in 1823
2021    625
Name: year, dtype: int64
1178 in 1823
2020    2086
Name: year, dtype: int64
1179 in 1823
2020    24
Name: year, dtype: int64
1180 in 1823
2019    2114
Name: year, dtype: int64
1181 in 1823
2021    1018
Name: year, dtype: int64
1182 in 1823
2020    741
Name: year, dtype: int64
1183 in 1823
2019    3
Name: year, dtype: int64
1184 in 1823
2020    1836
Name: year, dtype: int64
1185 in 1823
2021    2050
Name: year, dtype: int64
1186 in 1823
2020    818
Name: year, dtype:

1330 in 1823
2021    759
Name: year, dtype: int64
1331 in 1823
2019    76
Name: year, dtype: int64
1332 in 1823
2021    257
Name: year, dtype: int64
1333 in 1823
2021    1835
Name: year, dtype: int64
1334 in 1823
2020    2710
Name: year, dtype: int64
1335 in 1823
2020    1433
Name: year, dtype: int64
1336 in 1823
2020    1325
Name: year, dtype: int64
1337 in 1823
2021    6213
Name: year, dtype: int64
1338 in 1823
2019    750
Name: year, dtype: int64
1339 in 1823
2020    3561
Name: year, dtype: int64
1340 in 1823
2021    1228
Name: year, dtype: int64
1341 in 1823
2020    1
Name: year, dtype: int64
1342 in 1823
2019    1465
Name: year, dtype: int64
1343 in 1823
2021    2772
Name: year, dtype: int64
1344 in 1823
2021    873
Name: year, dtype: int64
1345 in 1823
2021    643
Name: year, dtype: int64
1346 in 1823
2020    3600
Name: year, dtype: int64
1347 in 1823
2020    2380
Name: year, dtype: int64
1348 in 1823
2020    56
Name: year, dtype: int64
1349 in 1823
2020    2926
Name: year, dtype

1493 in 1823
2020    383
Name: year, dtype: int64
1494 in 1823
2021    17
Name: year, dtype: int64
1495 in 1823
2021    1887
Name: year, dtype: int64
1496 in 1823
2021    1005
Name: year, dtype: int64
1497 in 1823
2019    876
Name: year, dtype: int64
1498 in 1823
2019    2333
Name: year, dtype: int64
1499 in 1823
2020    356
Name: year, dtype: int64
1500 in 1823
2019    7676
Name: year, dtype: int64
1501 in 1823
2019    5875
Name: year, dtype: int64
1502 in 1823
2020    3689
Name: year, dtype: int64
1503 in 1823
2020    148
Name: year, dtype: int64
1504 in 1823
2021    7628
Name: year, dtype: int64
1505 in 1823
2021    3810
Name: year, dtype: int64
1506 in 1823
2020    3534
Name: year, dtype: int64
1507 in 1823
2020    693
Name: year, dtype: int64
1508 in 1823
2019    5
Name: year, dtype: int64
1509 in 1823
2020    1289
Name: year, dtype: int64
1510 in 1823
2020    4717
Name: year, dtype: int64
1511 in 1823
2021    941
Name: year, dtype: int64
1512 in 1823
2020    1234
Name: year, dtyp

1656 in 1823
2021    409
Name: year, dtype: int64
1657 in 1823
2020    239
Name: year, dtype: int64
1658 in 1823
2019    118
Name: year, dtype: int64
1659 in 1823
2021    390
Name: year, dtype: int64
1660 in 1823
2019    2396
Name: year, dtype: int64
1661 in 1823
2019    1694
Name: year, dtype: int64
1662 in 1823
2020    2667
Name: year, dtype: int64
1663 in 1823
2021    105
Name: year, dtype: int64
1664 in 1823
2021    2489
Name: year, dtype: int64
1665 in 1823
2019    1858
Name: year, dtype: int64
1666 in 1823
2021    25
Name: year, dtype: int64
1667 in 1823
2020    3779
Name: year, dtype: int64
1668 in 1823
2020    733
Name: year, dtype: int64
1669 in 1823
2021    1458
Name: year, dtype: int64
1670 in 1823
2020    2302
Name: year, dtype: int64
1671 in 1823
2021    411
Name: year, dtype: int64
1672 in 1823
2019    2097
Name: year, dtype: int64
1673 in 1823
2019    1409
Name: year, dtype: int64
1674 in 1823
2020    1214
Name: year, dtype: int64
1675 in 1823
2020    1206
Name: year, dt

1819 in 1823
2020    7832
Name: year, dtype: int64
1820 in 1823
2019    1760
Name: year, dtype: int64
1821 in 1823
2021    3992
Name: year, dtype: int64
1822 in 1823
2020    904
Name: year, dtype: int64
number of observtaions for 2019, 2020, 2021: 663268 999290 930799


In [80]:
##directly join the l2b metrics with the tile 173060 level l4a smaples 
tileEx= pd.read_csv('ls_ltr_training/biome1_l4a_topo_ls_lhs_nbr_33_dur8_highval.csv')
print(tileEx.shape)
tileEx['shot_number'].duplicated().value_counts()

data = [l2b_samps2019, l2b_samps2020, l2b_samps2021]
l2b_samps_all= pd.concat(data)

tileEx2 =tileEx.drop_duplicates()
tileEx2.shape

tileEx3= tileEx.drop_duplicates(subset=['shot_number'])
tileEx3.shape

tile_samps_l2bl4a = tileEx3.merge(l2b_samps_all,on='shot_number',how='left')
print(tile_samps_l2bl4a.shape)

(2432320, 234)


## Step 5 left join with the level 4a data to retrive the original l4a attributes

In [82]:
subSamps=[tileSamps, tileSamps2, tileSamps3]#pd.read_csv('samples/all_gedi_tile173060plus3.csv')
allSamps = pd.concat(subSamps)

print(allSamps.shape)
tt= allSamps[~allSamps['agbd'].isna()]  #822686
tt2= allSamps.loc[(allSamps['algorithm_run_flag']==1) & (allSamps['l2_quality_flag']==1) &
#                           (alldf['degrade_flag']==0) &
#                             (tileSamps['biome']==1)&
#                            (df['biome']!=13) & (df['biome']!=14) & (df['biome']!=0) &
                          (allSamps['l4_quality_flag']==1) & allSamps['agbd'].notnull()]
print(len(tt), len(tt2))

(2649875, 52)
2649875 2649875


In [83]:
tile_samps_l2bl4a_2 = tile_samps_l2bl4a.merge(tt2,on='shot_number',how='left')
print(tile_samps_l2bl4a_2.shape)

(2432320, 285)


In [84]:
#key quality filtre first applied here 
tile_samps_l2bl4a_2['agbd']  = tile_samps_l2bl4a_2['agbd_x']
tile_samps_l2bl4a_2=  tile_samps_l2bl4a_2.loc[(tile_samps_l2bl4a_2['l2a_quality_flag']==1) &
                                    (tile_samps_l2bl4a_2['l2b_quality_flag']==1) &
                                    (tile_samps_l2bl4a_2['geolocation_degrade_flag']==0) &
                          (tile_samps_l2bl4a_2['l4_quality_flag']==1) & tile_samps_l2bl4a_2['agbd'].notnull()]

print(tile_samps_l2bl4a_2.shape)

(1679042, 286)


In [85]:
tile_samps_l2bl4a_2.to_csv('ls_ltr_training/biome1_l4a_topo_ls_lhs_nbr_33_dur8_highval_filtered.csv')

## From now on go to step 8a and 8b (skip extraction and start w/ modelling step) for tile level model training and fitting